In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
import torch
from tqdm import tqdm
import pandas as pd
import pickle

In [ ]:
%run data_preprocessing.ipynb

In [ ]:
df = pd.read_csv('tweets_prepro.csv')

In [ ]:
import torch

# Step 1: Tokenize and encode in one go
encoded_sentence = tokenizer.encode("J'aime le camembert !", return_tensors='pt')

# encoded_sentence is now a tensor of shape (1, sequence_length)

# Step 2: Pass through CamemBERT
outputs = camembert(encoded_sentence)

# Step 3: Extract token embeddings
embeddings = outputs.last_hidden_state  # shape: (1, seq_len, hidden_dim)


In [ ]:
#sentence_embedding = embeddings.mean(dim=1)  # average over tokens

In [ ]:
embeddings.shape

In [ ]:
annotated_corpus = pd.read_csv('corpus_SexistContent.csv', sep='\t', header=None, names=['tweet_id', 'label'])
df_whole = pd.merge(df, annotated_corpus, on = 'tweet_id')

In [ ]:
import pandas as pd
import torch
import numpy as np
import pickle
from transformers import CamembertTokenizer, CamembertModel

# Load model and tokenizer
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base")

# Dictionary to store everything
embeddings_dict = {}

# Process each row
for _, row in tqdm(df_whole.iterrows()):
    sentence = row['text_clean']
    label = row['label']
    
    # Tokenize and encode sentence (with special tokens)
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=False)
    
    with torch.no_grad():
        outputs = model(**inputs)
        # Extract per-token embeddings (shape: [1, seq_len, 768] -> [seq_len, 768])
        token_embeddings = outputs.last_hidden_state.squeeze(0).numpy()
    
    # Save to dictionary
    embeddings_dict[sentence] = {
        "embeddings": token_embeddings,  # shape: (num_tokens, 768)
        "label": label
    }

# Save to .pkl
with open("labeled_token_embeddings.pkl", "wb") as f:
    pickle.dump(embeddings_dict, f)


In [ ]:
embeddings_dict[df.loc[0,'text_clean']]